<a href="https://colab.research.google.com/github/keisukecl/llm-detect-ai-generated-text/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sys
import os
import shutil
from requests import get

pd.set_option('display.max_columns', None)

In [ ]:
class Config:
    name = "EDA"
    # Colab Env
    upload_from_colab = True
    drive_path = "/content/drive/MyDrive/llm-detect-ai-generated-text"

    # Kaggle Env
    kaggle_dataset_path = None

    debug = False

In [ ]:
COLAB = "google.colab" in sys.modules

In [ ]:
if COLAB:
    print("This environment is Google Colab")

    # mount
    from google.colab import drive
    if not os.path.isdir("/content/drive"):
        drive.mount('/content/drive')

    # set dirs
    DRIVE = Config.drive_path
    EXP = (Config.name if Config.name is not None
           else get("http://172.28.0.2:9000/api/sessions").json()[0]["name"][:-6])
    INPUT = os.path.join(DRIVE, "Input")
    OUTPUT = os.path.join(DRIVE, "Output")
    SUBMISSION = os.path.join(DRIVE, "Submission")
    OUTPUT_EXP = os.path.join(OUTPUT, EXP)
    EXP_MODEL = os.path.join(OUTPUT_EXP, "model")
    EXP_FIG = os.path.join(OUTPUT_EXP, "fig")
    EXP_PREDS = os.path.join(OUTPUT_EXP, "preds")

    # make dirs
    for d in [INPUT, SUBMISSION, EXP_MODEL, EXP_FIG, EXP_PREDS]:
        os.makedirs(d, exist_ok=True)

else:
  print("This environment is Kaggle Kernel")

  # set dirs
  INPUT = "../Input/llm-detect-ai-generated-text"
  EXP, OUTPUT, SUBMISSION = "./", "./", "./"
  EXP_MODEL = os.path.join(EXP, "model")
  EXP_FIG = os.path.join(EXP, "fig")
  EXP_PREDS = os.path.join(EXP, "preds")

  # copy dirs
  if Config.kaggle_dataset_path is not None:
      KD_MODEL = os.path.join(Config.kaggle_dataset_path, "model")
      KD_EXP_PREDS = os.path.join(Config.kaggle_dataset_path, "preds")
      shutil.copytree(KD_MODEL, EXP_MODEL)
      shutil.copytree(KD_EXP_PREDS, EXP_PREDS)

  # make dirs
  for d in [EXP_MODEL, EXP_FIG, EXP_PREDS]:
      os.makedirs(d, exist_ok=True)

This environment is Google Colab
Mounted at /content/drive


In [ ]:
# load data
train_essays = pd.read_csv(os.path.join(INPUT, "train_essays.csv" if COLAB else "train_logs.csv"))
train_prompts = pd.read_csv(os.path.join(INPUT, "train_prompts.csv" if COLAB else "train_scores.csv"))
test = pd.read_csv(os.path.join(INPUT, "test_essays.csv"))
#sample_submission = pd.read_csv(os.path.join(INPUT, "sample_submission.csv"))


In [ ]:
train_essays.shape

(1378, 4)

In [ ]:
train_prompts.shape

(2, 4)

In [ ]:
train_prompts

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...
